[필사 원본 링크](https://www.kaggle.com/code/sasha18/recommend-books-using-count-tfidf-on-titles) <br>
[코사인 유사도 관련 영상](https://www.youtube.com/watch?v=e9U0QAFbfLI&ab_channel=StatQuestwithJoshStarmer)
## 이 노트북을 선택한 이유
* 제목에 TFidf 가 눈에 가장 먼저 들어왔다.
* 관련 기억이 전혀 나지 않음
* 복습하기에 좋은 노트북이라고 판단

## Import packages and dataset

In [2]:
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

data = pd.read_csv("/Users/jahyeon_gu/Downloads/books.csv")
data.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


## 데이터 살펴보기 (프리스타일)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10000 non-null  int64  
 1   book_id                    10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

### 수치형 변수

In [4]:
data.describe()

,id,book_id,best_book_id,work_id,books_count,isbn13,original_publication_year,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
count,10000.00000,1.000000e+04,1.000000e+04,1.000000e+04,10000.000000,9.415000e+03,9979.000000,10000.000000,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04
mean,5000.50000,5.264697e+06,5.471214e+06,8.646183e+06,75.712700,9.755044e+12,1981.987674,4.002191,5.400124e+04,5.968732e+04,2919.955300,1345.040600,3110.885000,11475.893800,1.996570e+04,2.378981e+04
std,2886.89568,7.575462e+06,7.827330e+06,1.175106e+07,170.470728,4.428619e+11,152.576665,0.254427,1.573700e+05,1.678038e+05,6124.378132,6635.626263,9717.123578,28546.449183,5.144736e+04,7.976889e+04
min,1.00000,1.000000e+00,1.000000e+00,8.700000e+01,1.000000,1.951703e+08,-1750.000000,2.470000,2.716000e+03,5.510000e+03,3.000000,11.000000,30.000000,323.000000,7.500000e+02,7.540000e+02
25%,2500.75000,4.627575e+04,4.791175e+04,1.008841e+06,23.000000,9.780316e+12,1990.000000,3.850000,1.356875e+04,1.543875e+04,694.000000,196.000000,656.000000,3112.000000,5.405750e+03,5.334000e+03
50%,5000.50000,3.949655e+05,4.251235e+05,2.719524e+06,40.000000,9.780452e+12,2004.000000,4.020000,2.115550e+04,2.383250e+04,1402.000000,391.000000,1163.000000,4894.000000,8.269500e+03,8.836000e+03
75%,7500.25000,9.382225e+06,9.636112e+06,1.451775e+07,67.000000,9.780831e+12,2011.000000,4.180000,4.105350e+04,4.591500e+04,2744.250000,885.000000,2353.250000,9287.000000,1.602350e+04,1.730450e+04
max,10000.00000,3.328864e+07,3.553423e+07,5.639960e+07,3455.000000,9.790008e+12,2017.000000,4.820000,4.780653e+06,4.942365e+06,155254.000000,456191.000000,436802.000000,793319.000000,1.481305e+06,3.011543e+06


### 범주형 변수

In [5]:
data.describe(include="O")

,isbn,authors,original_title,title,language_code,image_url,small_image_url
count,9300,10000,9415,10000,8916,10000,10000
unique,9300,4664,9274,9964,25,6669,6669
top,439023483,Stephen King,,Selected Poems,eng,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
freq,1,60,5,4,6341,3332,3332


## Data Preprocessing

In [9]:
books_title = data[['book_id', 'title']]

# 책의 개수 : 10,000개 
print(books_title.shape)
books_title.head()

(10000, 2)


,book_id,title
0,2767052,"The Hunger Games (The Hunger Games, #1)"
1,3,Harry Potter and the Sorcerer's Stone (Harry P...
2,41865,"Twilight (Twilight, #1)"
3,2657,To Kill a Mockingbird
4,4671,The Great Gatsby


## CountVectorizer

In [10]:
vect = CountVectorizer(analyzer='word', ngram_range=(1, 2), stop_words='english', min_df=0.002)

vect.fit(books_title["title"])
title_matrix = vect.transform(books_title["title"])
title_matrix.shape

(10000, 261)

In [11]:
# 원본에는 
# vect.get_feature_names() 사용
# 1.0 버전 부터 get_feature_names_out() 로 대체
features = vect.get_feature_names_out()

# 261개의 단어
print(len(features))
features

261


array(['01', '10', '11', '12', '13', '14', '15', '16', '39', 'adventures',
       'alex', 'alex cross', 'america', 'american', 'angel', 'angels',
       'anita', 'anita blake', 'apprentice', 'art', 'bad', 'batman',
       'beautiful', 'beauty', 'best', 'big', 'black', 'blake',
       'blake vampire', 'blood', 'blue', 'body', 'bone', 'bones', 'book',
       'books', 'born', 'bosch', 'bosch universe', 'boy', 'boys',
       'broken', 'brothers', 'case', 'cat', 'child', 'children',
       'chronicles', 'circle', 'city', 'club', 'complete', 'confessions',
       'cross', 'cycle', 'dance', 'dark', 'dark hunter', 'darkest',
       'darkness', 'daughter', 'davenport', 'dawn', 'day', 'days', 'dead',
       'death', 'death death', 'detective', 'devil', 'diaries', 'diary',
       'die', 'discworld', 'dog', 'don', 'dragon', 'dream', 'dreams',
       'earth', 'empire', 'end', 'fall', 'fallen', 'family', 'fear',
       'files', 'food', 'forever', 'forgotten', 'forgotten realms',
       'game', 'gard

## Cosine Similarity between Titles
Things to do:

* Initialize Cosine similarity into title matrix
* Extract features from book title
* Using Cosine similarity between this title and all other titles to be recommended the top 10 books

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim_titles = cosine_similarity(title_matrix, title_matrix)
cosine_sim_titles.shape

(10000, 10000)

In [14]:
title_id = 100
books_title['title'].iloc[title_id]

'Me Talk Pretty One Day'

In [15]:
feature_array = np.squeeze(title_matrix[title_id].toarray())
feature_array


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
idx = np.where(feature_array > 0)
idx

(array([ 63, 179]),)

In [17]:
[features[x] for x in idx[0]]

['day', 'pretty']

In [18]:
# Find index of feature
idx[0]

array([ 63, 179])

In [19]:
n = 15
top_n_idx = np.flip(np.argsort(cosine_sim_titles[title_id, ]), axis=0)[0:n]
top_n_sim_values = cosine_sim_titles[title_id, top_n_idx]
top_n_sim_values

array([1.        , 0.70710678, 0.70710678, 0.70710678, 0.70710678,
       0.70710678, 0.70710678, 0.70710678, 0.70710678, 0.70710678,
       0.70710678, 0.70710678, 0.70710678, 0.70710678, 0.70710678])

In [29]:
arr = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

print(np.array(arr))

[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [49]:
print(np.flip(arr, 0))
print("-" * 10)
print(np.flipud(arr))
print("-" * 10)
print(np.flip(arr, -2))
print("-" * 10)
print(np.flip(arr, (0)))

[[7 8 9]
 [4 5 6]
 [1 2 3]]
----------
[[7 8 9]
 [4 5 6]
 [1 2 3]]
----------
[[7 8 9]
 [4 5 6]
 [1 2 3]]
----------
[[7 8 9]
 [4 5 6]
 [1 2 3]]


In [48]:
print(np.flip(arr, 1))
print("-" * 10)
print(np.fliplr(arr))
print("-" * 10)
print(np.flip(arr, -1))
print("-" * 10)
print(np.flip(arr, (1)))

[[3 2 1]
 [6 5 4]
 [9 8 7]]
----------
[[3 2 1]
 [6 5 4]
 [9 8 7]]
----------
[[3 2 1]
 [6 5 4]
 [9 8 7]]
----------
[[3 2 1]
 [6 5 4]
 [9 8 7]]


In [47]:
# The default, axis=None, will flip over all of the axes of the input array
print(np.flip(arr))
print("-" * 10)
print(np.flip(arr, (0, 1)))

[[9 8 7]
 [6 5 4]
 [3 2 1]]
----------
[[9 8 7]
 [6 5 4]
 [3 2 1]]


In [42]:
print(np.flip(arr, -2))

[[7 8 9]
 [4 5 6]
 [1 2 3]]


In [46]:
print(np.flip(arr, (0, 1)))

[[9 8 7]
 [6 5 4]
 [3 2 1]]
